# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
ls

CONTRIBUTING.md       README.md             index.ipynb
LICENSE.md            homepage_actions.csv*


In [16]:
#Your code here
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

In [3]:
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [4]:
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [5]:
df_view = df[df['action'] == 'view']['id'].unique()
df_click = df[df['action'] == 'click']['id'].unique()

In [6]:
# How many viewers also clicked?
counter = 0
for x in df_view:
    if x in df_click:
        counter += 1
counter

1860

In [7]:
# Using set
df_view_set = set(df_view)
df_click_set = set(df_click)
len(df_view_set.intersection(df_click_set))


1860

In [8]:
# Did anyone clicked that didn't view?
df_click_set - df_view_set

set()

In [9]:
# Is there any overlap between the control and experiment groups?
exp_group = df[df['group'] == 'experiment']['id'].unique()
cont_group = df[df['group'] == 'control']['id'].unique()

In [10]:
exp_group_set = set(exp_group)
cont_group_set = set(cont_group)
len(cont_group_set.intersection(exp_group_set))

0

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [23]:
exp_group = df[df['group'] == 'experiment'].copy()
exp_group['dummy'] = 1
experiment = exp_group.pivot(index = 'id', columns = 'action', values = 'dummy')
experiment.fillna(0, inplace = True)
experiment.head()

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0


In [24]:
cont_group = df[df['group'] == 'control'].copy()
cont_group['dummy'] = 1
control = cont_group.pivot(index = 'id', columns = 'action', values = 'dummy')
control.fillna(0, inplace = True)
control.head()

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [26]:
print(len(experiment))
print(len(control))


2996
3332


In [31]:
stats.ttest_ind(control['click'], experiment['click'], equal_var = False )

Ttest_indResult(statistic=-2.615440020788211, pvalue=0.008932805628674201)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [40]:
#Your code here
expected_numbers_of_clicks = np.round(len(experiment) * control['click'].mean(), 1)
expected_numbers_of_clicks

838.0

In [42]:
actual = experiment['click'].sum()
actual

928.0

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [37]:
#Your code here
n = len(experiment)
p = control['click'].mean()
var = n * p * (1 - p)
std = np.sqrt(var)
std

24.568547907005815

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [44]:
#Your code here
z = (actual - expected_numbers_of_clicks) / std
z

3.6632201602088235

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [46]:
from scipy.stats import norm

In [47]:
norm.sf(z)

0.0001245321456477967

In [ ]:
# The p val for z is very similar to the p val of ttest.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.